In [1]:
import requests # Envoyer des requêtes, Ouvrir un apge Web
from bs4 import BeautifulSoup # Parser du HTML

In [2]:
msn = requests.get("https://www.msn.com/en-us/money/markets/currencies")
print( "code d'état :", msn.status_code)

code d'état : 200


In [3]:
soup = BeautifulSoup(msn.text)

In [4]:
currency_class = ".mjrcurrncsitem"

In [5]:
# cur = soup.find( class_ =  currency_class) 
cur = soup.select_one( currency_class) 
# Retenez cette syntaxe, elle permet d'extraire une classe quelconque
if cur and len(cur) :
  print("Bravo: il semblerait que j'ai trouvé la classe!")
else:
  print("Je n'ai toujours pas trouvé la classe mère")

Bravo: il semblerait que j'ai trouvé la classe!


In [6]:
thead_class = '.mjrcurrncsrow.tblheaderrow'

In [15]:
header = cur.select_one(thead_class)
headers = header.select(".mctblheading") # Avez-vous pu retrouver la classe `mctblheading`  de vous mêmes ?
header_values = []
for header_element in headers :
#     header_values.append(header_element.select_one("p").attrs["title"])
    header_values.append(header_element.select_one("p[title]").attrs["title"])
print(header_values)

['Major Currencies', 'Price', 'Change', 'Change%', '52 Week High', '52 Week Low']


# Corps de la table

In [17]:
trows_class = ".mjcurrncs-data"

In [18]:
rows_container = cur.select_one(trows_class)
rows = rows_container.select( ".mcrow")
row = rows[1]
print(row)

<div class="mcrow">
<span aria-hidden="True" class="flagcol"><span class="cntryflag" id="JPN"> </span></span>
<div class="cntrycnvrsn">
<span class="cntrycol"><p class="truncated-string" title="Japanese Yen">Japanese Yen</p></span>
<span class="cnvrsncol"><p class="truncated-string" title="USD/JPY">USD/JPY</p></span>
</div>
<span class="pricecol"><p aria-label="Price 110.0100" class="truncated-string" title="110.0100">110.0100</p></span>
<div class="chcpclub">
<span class="chngcol chng-value increase"><p aria-label="Change +0.0800" class="truncated-string" title="+0.0800">+0.0800</p></span>
<span class="chpcol chp-value increase"><p aria-label="Change% +0.0728%" class="truncated-string" title="+0.0728%">+0.0728%</p></span>
</div>
<span class="rng52wkcol"><p aria-label="52 Week High 112.3900" class="truncated-string" title="112.3900">112.3900</p></span>
<span class="rng52wkcol"><p aria-label="52 Week Low 104.4400" class="truncated-string" title="104.4400">104.4400</p></span>
</div>


In [19]:
# Cette fonction extraie les différentes colonnes d'une ligen quelconque

def get_row(row):
    flag = row.select_one( ".cntryflag").attrs["id"]
    
    cur_names = row.select_one( ".cntrycnvrsn")
#     cur_name1 =  cur_names.select_one(  ".cntrycol").p.attrs["title"]
    cur_name1 =  cur_names.select_one(  ".cntrycol p").attrs["title"]
#     cur_name2 =  cur_names.find( class_ = "cnvrsncol").find("p").attrs["title"]
    cur_name2 =  cur_names.select_one(  ".cnvrsncol p").attrs["title"]
    
    price = row.select_one(".pricecol").select_one("p").attrs["title"]
    
    changes = row.select_one(".chcpclub")
    chg_absolute = changes.select_one( ".chngcol p").attrs["title"]
    chg_perc  = changes.select_one(  ".chpcol p").attrs["title"]
    
    return {"flag": flag, "cur_name1": cur_name1, "cur_name2":cur_name2,  "price": price, 
            "chg_absolute": chg_absolute,  "chg_perc": chg_perc}

In [20]:
get_row(row)

{'flag': 'JPN',
 'cur_name1': 'Japanese Yen',
 'cur_name2': 'USD/JPY',
 'price': '110.0100',
 'chg_absolute': '+0.0800',
 'chg_perc': '+0.0728%'}

In [21]:
data = [get_row(row) for row in rows]
data[:2]

[{'flag': 'EUR',
  'cur_name1': 'Euro',
  'cur_name2': 'USD/EUR',
  'price': '0.8982',
  'chg_absolute': '+0.0003',
  'chg_perc': '+0.0334%'},
 {'flag': 'JPN',
  'cur_name1': 'Japanese Yen',
  'cur_name2': 'USD/JPY',
  'price': '110.0100',
  'chg_absolute': '+0.0800',
  'chg_perc': '+0.0728%'}]

In [22]:
import pandas as pd

In [23]:
df = pd.DataFrame(data)
df.head()

,flag,cur_name1,cur_name2,price,chg_absolute,chg_perc
0,EUR,Euro,USD/EUR,0.8982,+0.0003,+0.0334%
1,JPN,Japanese Yen,USD/JPY,110.0100,+0.0800,+0.0728%
2,GBR,British Pound,GBP/USD,1.2968,-0.0020,-0.1540%
3,CAN,Canadian Dollar,USD/CAD,1.3074,+0.0018,+0.1379%
4,AUS,Australian Dollar,USD/AUD,1.4497,+0.0013,+0.0898%
